In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.layers import SimpleRNN
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Подготовка датасета

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/medium_articles.csv")
print(data)

                                            title  \
0                             Mental Note Vol. 24   
1                       Your Brain On Coronavirus   
2                                  Mind Your Nose   
3                        The 4 Purposes of Dreams   
4                Surviving a Rod Through the Head   
...                                           ...   
192363        Why do you need a cleaning service?   
192364  Daily cleaning and maintenance of bedding   
192365        Beneficial Advice on Bond Cleaning!   
192366    How I Learned Romanian in 37 Easy Steps   
192367     Trying Pimsleur Cantonese in Hong Kong   

                                                     text  \
0       Photo by Josh Riemer on Unsplash\n\nMerry Chri...   
1       Your Brain On Coronavirus\n\nA guide to the cu...   
2       Mind Your Nose\n\nHow smell training can chang...   
3       Passionate about the synergy between science a...   
4       You’ve heard of him, haven’t you? Phineas Gage... 

In [ ]:
from collections import Counter
def pars_tags (data):
  d = {}
  for el in data['tags']:
    for i in el:
      if i in d:
        d[i] += 1
      else:
        d[i] = 1
  cnt = Counter(d).most_common(10)
  return cnt

In [ ]:
def upd_data (data, tag_list):
  new_data = pd.DataFrame()
  for _, el in data.iterrows():
    for i in el['tags']:
      for j in tag_list:
        if i == j[0]:
          el['tags'] = i
          new_data = new_data.append(el, ignore_index = True)
  return new_data  

In [ ]:
tmp = data.tags.apply(lambda x: x[1:-1].split(','))
tmp_data = data.copy() 
tmp_data['tags'] = tmp
tags_list = pars_tags(tmp_data)
new_data = upd_data(tmp_data, tags_list)
print(new_data)

<ipython-input-5-2eb8360fb675>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(el, ignore_index = True)


                                                   title  \
0      How to Turn Your Popular Blog Series Into a Be...   
1              Four Exercises to Strengthen Your Writing   
2      This 10-Minute Routine Will Increase Your Clar...   
3              How to Make Your Day Job Support Your Art   
4      An Effective Five-Step Process for Writing Cap...   
...                                                  ...   
50163               bStable needs you 🦸‍♂️ 🧑‍✈️🧑‍🔬🧑‍💻👨‍🌾   
50164               bStable needs you 🦸‍♂️ 🧑‍✈️🧑‍🔬🧑‍💻👨‍🌾   
50165                Status messenger for Desktop review   
50166  The Impact of Technology on Hotel Guest Experi...   
50167  (Live — Streaming) : 2021 Soul Train Music Awards   

                                                    text  \
0      How to Turn Your Popular Blog Series Into a Be...   
1      A few months ago, I wrote an article sharing s...   
2      “Your subconscious mind works continuously, wh...   
3      Gather around kids and let me te

In [ ]:
text_data = open('Articles.txt', 'w')
for idx, item in new_data.iterrows():
  article = item["text"]
  text_data.write(article)
text_data.close()

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Тренировка модели


In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [ ]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

In [ ]:
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=mlm,
    )
    return data_collator

In [ ]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)
      
  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )
      
  trainer.train()
  trainer.save_model()

In [ ]:
train_file_path = "Articles.txt"
model_name = 'gpt2'
output_dir = '/content/drive/MyDrive/result'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 5.0
save_steps = 500

In [ ]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.29.2
Uninstalling transformers-4.29.2:
  Successfully uninstalled transformers-4.29.2
Found existing installation: accelerate 0.19.0
Uninstalling accelerate-0.19.0:
  Successfully uninstalled accelerate-0.19.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  Using cached accelerate-0.19.0-py3-none-any.whl (219 kB)


In [ ]:
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## Вывод

In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [ ]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer


def generate_text(sequence, max_length):
    model_path = "/content/drive/MyDrive/result"
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [ ]:
sequence = 'Paper is very important'
max_len = 100
generate_text(sequence, max_len)

Paper is very important. It is crucial that you remember this.

Remember to choose a story and character that fits your subject. It helps to have some of your best ideas, too.

The key to success with Medium’s content strategy is writing well. The goal here is to rank as high as possible and stay there for at least one day. Be creative. Be creative about not being too specific.

5. Pick a story

The most important thing you
